In [1]:
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

### Analyze Dataset

In [2]:
df = pd.read_csv('dataset/100k_a.csv')

In [3]:
df.head()

,1,33842865744,mithrain,154,156
0,1,33846768288,alptv,166,169
1,1,33886469056,mithrain,587,588
2,1,33887624992,wtcn,589,591
3,1,33890145056,jrokezftw,591,594
4,1,33903958784,berkriptepe,734,737


In [4]:
df = pd.read_csv('dataset/100k_a.csv', names=['user_id', 'stream_id', 'streamer_name', 'start_time', 'stop_time'])

In [5]:
df.head()

,user_id,stream_id,streamer_name,start_time,stop_time
0,1,33842865744,mithrain,154,156
1,1,33846768288,alptv,166,169
2,1,33886469056,mithrain,587,588
3,1,33887624992,wtcn,589,591
4,1,33890145056,jrokezftw,591,594


In [6]:
df.shape

(3051733, 5)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3051733 entries, 0 to 3051732
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   user_id        int64 
 1   stream_id      int64 
 2   streamer_name  object
 3   start_time     int64 
 4   stop_time      int64 
dtypes: int64(4), object(1)
memory usage: 116.4+ MB


In [8]:
df.describe()

,user_id,stream_id,start_time,stop_time
count,3.051733e+06,3.051733e+06,3.051733e+06,3.051733e+06
mean,5.032313e+04,3.413002e+10,3.146044e+03,3.149186e+03
std,2.889476e+04,1.682242e+08,1.772520e+03,1.772656e+03
min,1.000000e+00,3.380153e+10,0.000000e+00,1.000000e+00
25%,2.528400e+04,3.398884e+10,1.618000e+03,1.620000e+03
50%,5.029900e+04,3.413069e+10,3.174000e+03,3.178000e+03
75%,7.545200e+04,3.427399e+10,4.676000e+03,4.679000e+03
max,1.000000e+05,3.441642e+10,6.147000e+03,6.148000e+03


In [9]:
df['streamer_name'].value_counts()

ninja             45144
tfue              40136
shroud            27362
riotgames         17633
sodapoppin        14590
                  ...  
thescruffman92        1
imbeergoggles         1
crhuou                1
dark5sky              1
temptedtrigger        1
Name: streamer_name, Length: 162625, dtype: int64

### Split Train, Test & Validation Sets

In [10]:
# همین اول کاری باید دیتای آموزش رو از تست جدا کنیم. چون با بازه های زمانی سر و کار داریم و کل دیتا در 43 روز جمع شده،
# فکر می کنم که خوب باشه بیایم مثلا 5 روز آخر رو بعنوان تست در نظر بگیریم و قبل اون رو به دیتای آموزش اختصاص بدیم

0 - 1 &#8594; 10 min<br>
0 - 2 &#8594; 20 min<br>
0 - 3 &#8594; 30 min<br>
0 - 4 &#8594; 40 min<br>
0 - 5 &#8594; 50 min<br>
0 - 6 &#8594; 60 min<br>
.<br>
.<br>
.<br>
0 - 6148 &#8594; 61480 min<br><hr>
1 week &#8594; 7 days &#8594; 7 × 24 hours &#8594; 7 × 24 × 60 minutes &#8594; 10080 min<br><hr>
week 1 &#8594; 0000 - 1008 &#8594; 10080 min<br>
week 2 &#8594; 1008 - 2016 &#8594; 10080 min<br>
week 3 &#8594; 2016 - 3024 &#8594; 10080 min<br>
week 4 &#8594; 3024 - 4032 &#8594; 10080 min<br>
week 5 &#8594; 4032 - 5040 &#8594; 10080 min<br>
week 6 &#8594; 5040 - 6148 &#8594; 11080 min<br><hr>
Train Set &#8594; 0000 - 5420 &#8594; 54200 min &#8594; 903.33 hours &#8594; 37.64 days<br>
Test Set &#8594; 5420 - 6140 &#8594; 7200 min &#8594; 120 hours &#8594; 5 days <br>

In [11]:
train = df[df['start_time'] < 5420]
test = df[(df['start_time'] >= 5420)]

In [12]:
print(train.shape[0]/df.shape[0])
print(test.shape[0]/df.shape[0])

0.8725438955504954
0.1274561044495046


### Extract Features

In [13]:
# اولین چیزی که به نظرم میرسه اینکه بیایم ببینیم هر کاربر چه مدت زمانی در هر استریم فعالیت داشته

train['user_stream_du'] = train['stop_time'] - train['start_time']

In [14]:
# یه چیزی رو هم همین اول کار باید بفهمیم، اونم اینکه فعالیت کل کاربرها در هر استریم در دیتاستی که داریم در کل چقدر طول کشیده
# برای این کار باید بیایم برای هر استریم ببینیم که کوچک ترین زمان شروع و بزرگ ترین زمان پایان چی بوده و اختلافشو حساب کنیم

train['stream_du'] = train.groupby(['stream_id'])['stop_time'].transform('max') - train.groupby(['stream_id'])['start_time'].transform('min')

In [15]:
# اینم حساب کنیم که در هر استریم چند یوزر داشتن چت میکردن و فعال بودن

train['stream_total_user'] = train.groupby(['stream_id'])['user_id'].transform('count')

In [16]:
# مورد بعدی که به نظرم میرسه اینکه بیایم ببینیم هر کاربر از قبل مجموعا در استریم های هر کانال چقدر فعال بوده
# منطقا اولین باری که کاربر از هر کانال یک استریم رو در اون چت میکنه سابقه ش صفره و بعد در اثر گذر زمان مجموعش رو هر دفعه باید حساب کنیم

train = train.sort_values(by='start_time')
train['user_channel_du'] = train.groupby(['user_id', 'streamer_name'])['user_stream_du'].transform('cumsum') - train['user_stream_du']

In [17]:
# شاید بد نباشه بیایم این رو هم بررسی کنیم که هر کاربر از قبل در کل در چند تا استریم شرکت کرده

train['user_total_stream'] = train.groupby(['user_id'])['stream_id'].transform('cumcount') 

In [18]:
# اینم بررسی کنیم که هر کاربر از قبل از هر کانال در چند تا استریم شرکت کرده

train['user_total_stream_per_channel'] = train.groupby(['user_id', 'streamer_name'])['stream_id'].transform('cumcount') 

In [19]:
# اگه تعداد استریم های هر کانال رو هم داشته باشیم شاید بشه ازش استفاده کرد

train['channel_total_stream'] = (~train['stream_id'].duplicated()).groupby(train['streamer_name']).transform('cumsum') - 1

In [20]:
# شاید خوب باشه که بیایم این رو هم حساب کنیم که هر کانال مجموعا، استریم هاش چقدر طول کشیده

temp = train.drop_duplicates(subset=['stream_id'])
train['channel_total_stream_du'] = temp.groupby(['streamer_name'])['stream_du'].transform('cumsum') - train['stream_du']
train = train.sort_values(by=['streamer_name', 'stream_id', 'start_time'])
train['channel_total_stream_du'].fillna(method='ffill', inplace=True)

In [21]:
# اینو هم حساب کنیم که تعداد استریم های هر کانال از زمان اولین چت کاربر به بعد چند تا بوده. 
# چون مثلا یه کانالی 100 تا استریم داشته قبلا ولی یه یوزر نمیدونه که کلا همچنین کانالی هست و تا به اینجا 100 تا استریم گذشته رو از دست داده
# ولی از روزی که اومده تا اخرین روز مثلا 5 تا استریم برگزار شده و هر 5 تارو تو چت شرکت کرده

train['channel_total_stream_since_user_join'] = train[train['user_total_stream_per_channel']==0]['channel_total_stream']
train = train.sort_values(by=['user_id', 'streamer_name', 'start_time'])
train['channel_total_stream_since_user_join'].fillna(method='ffill', inplace=True)
train['channel_total_stream_since_user_join'] = train['channel_total_stream'] - train['channel_total_stream_since_user_join']

In [22]:
# مدت زمان استریم ها از زمانی که کاربر در اولین استریم از یه کانالی فعال بوده به بعد هم خوبه که داشته باشیم دلیلش مشابه فیچر قبلیه

train['channel_total_stream_du_since_user_join'] = train[train['user_channel_du']==0]['channel_total_stream_du']
train = train.sort_values(by=['user_id', 'streamer_name', 'start_time'])
train['channel_total_stream_du_since_user_join'].fillna(method='ffill', inplace=True)
train['channel_total_stream_du_since_user_join'] = train['channel_total_stream_du'] - train['channel_total_stream_du_since_user_join']

In [23]:
train = train.sort_index()
train.head()

,user_id,stream_id,streamer_name,start_time,stop_time,user_stream_du,stream_du,stream_total_user,user_channel_du,user_total_stream,user_total_stream_per_channel,channel_total_stream,channel_total_stream_du,channel_total_stream_since_user_join,channel_total_stream_du_since_user_join
0,1,33842865744,mithrain,154,156,2,23,84,0,0,0,1,28.0,0.0,0.0
1,1,33846768288,alptv,166,169,3,9,23,0,1,0,1,18.0,0.0,0.0
2,1,33886469056,mithrain,587,588,1,35,82,2,2,1,5,113.0,4.0,85.0
3,1,33887624992,wtcn,589,591,2,30,73,0,3,0,3,75.0,0.0,0.0
4,1,33890145056,jrokezftw,591,594,3,10,10,0,4,0,8,115.0,0.0,0.0


### Calculate Favorite Channel Score

In [24]:
# تا اینجا 10 تا فیچر استخراج کردیم که ممکنه بعضیاشو استفاده کنیم بعضیاشو هم نکنیم
# از اینجا به بعد باید یک مشخصه ای رو در نظر بگیریم که بهمون بگه که میزان تعامل هر کاربر با هر کانال چقدر هست
# این کار رو میشه بر اساس فیچرهایی که تا اینجا از کاربران و استریم ها و کانال ها به دست آوردیم انجام بدیم

<div dir="rtl">
تا به اینجا یک سری فیچر استخراج کردیم که به شرح زیرهست:<br>
    1. میزان فعالیت کاربر در هر استریم = user_stream_du
    <br>
    2. مدت زمان کل استریم = stream_du
    <br>
    3. تعداد کل کاربران شرکت کننده در چت در هر استریم = stream_total_user
    <br>
    4. مدت زمان فعال بودن هر کاربر در استریم های قبلی از کانالی که در حال حاضر در استریم آن فعال است = user_channel_du
    <br>
    5. تعداد کل استریم هایی که کاربر از قبل در آن شرکت کرده است = user_total_stream 
    <br>
    6. تعداد کل استریم هایی که هر کاربر از قبل در آن فعال بوده از کانالی که در حال حاضر در استریم آن فعال است = user_total_stream_per_channel 
    <br>
    7. تعداد کل استریم هایی که هر کانال از ابتدا تا به زمان استریم فعلی داشته است = channel_total_stream
    <br>
    8. مدت زمان کل استریم هایی که هر کانال از ابتدا تا به زمان استریم فعلی داشته است = channel_total_stream_du
    <br> 
    9. تعداد کل استریم های هر کانال از زمان اولین فعالیت کاربر در یکی از استریم ها = channel_total_stream_since_user_join
    <br>
    10. مدت زمان کل استریم های هر کانال از زمان اولین فعالیت کاربر در یکی از استریم ها = channel_total_stream_du_since_user_join
    <br><hr>
   برای اینکه یکم واضح تر بشه برای یک سطر از دیتا ست فیچرهارو توضیح میدم.
 
</div>

In [25]:
train.loc[5465]

user_id                                            197
stream_id                                  34135849920
streamer_name                               esl_brazil
start_time                                        3210
stop_time                                         3211
user_stream_du                                       1
stream_du                                           22
stream_total_user                                  110
user_channel_du                                      7
user_total_stream                                    3
user_total_stream_per_channel                        2
channel_total_stream                                46
channel_total_stream_du                          597.0
channel_total_stream_since_user_join              25.0
channel_total_stream_du_since_user_join          418.0
Name: 5465, dtype: object

<div dir="rtl">
    الان اینجا کاربر با آیدی 197 در یک استریم از کانال esl_brazil شرکت کرده.
    <br>
    کل فعالیتی که در چت داشته معادل با 1 اسنپ شات بوده که یعنی همون 10 دقیقه. 
    <br>
    کل استریم هم 22 اسنپ شات بوده که یعنی 220 دقیقه.
    <br>
    تعداد کل کاربرانی که در چت این استریم فعالیت داشتن 110 تا بودن.  
    <br>
    کاربر از قبل در چت استریم های کانال esl_brazil به مدت 7 اسنپ شات که همون 70 دقیقه هست فعالیت داشته که معادل هست با 2 استریم.
    <br>
    همچنین، کاربر در کل از قبل در 3 استریم از کانال های دیگه هم فعال بوده است.
    <br>
    کاربر در 2 استریم قبلی از این کانال هم شرکت کرده است.
    <br>
    کانال esl_brazil از ابتدا تا قبل از این استریم 46 تا استریم دیگه داشته است.
    <br>
    مدت زمان استریم های قبلی کانال 597 تا اسنپ شات بوده که معادل هست با 5970 دقیقه.
    <br>
    از زمانی که کاربر در یکی از استریم های این کانال شرکت کرده تا به اینجا 25 تا استریم از این کانال برگزار شده.
    <br>
    مدت زمان استریم های برگزار شده از زمانی که کاربر در اولین استریم از این کانال شرکت کرده برابر با 418 اسنپ شات هست که همون 4180 دقیقه است.
    <hr>
    الان دنبال یه مشخصه ای باید باشیم که بتونیم به کمکش تعریف کنیم که هر کاربر با توجه به استریم هایی که شرکت کرده در کل چقدر به کانال ارائه دهنده علاقه منده. بعدش بر اساس این علاقه مندی ها میتونیم کانال های جدیدی رو برای هر کاربر پیشنهاد بدیم.
</div>

<div dir="rtl">
شاید بشه میزان علاقه مندی هر کاربر به هر کانال رو به صورت زیر تعریف کرد:
    <br>
</div>
Channel Score = (C/B) + (A/E) + (C/D)

<div dir="rtl">
    <hr>
    A. مدت زمان فعال بودن هر کاربر در استریم های قبلی از کانالی که در حال حاضر در استریم آن فعال است = user_channel_du
    <br>
    B. تعداد کل استریم هایی که کاربر از قبل در آن شرکت کرده است = user_total_stream 
    <br>
    C. تعداد کل استریم هایی که هر کاربر از قبل در آن فعال بوده از کانالی که در حال حاضر در استریم آن فعال است = user_total_stream_per_channel 
    <br>
    D. تعداد کل استریم های هر کانال از زمان اولین فعالیت کاربر در یکی از استریم ها = channel_total_stream_since_user_join
    <br>
    E. مدت زمان کل استریم های هر کانال از زمان اولین فعالیت کاربر در یکی از استریم ها = channel_total_stream_du_since_user_join
<!--     F. میزان فعالیت کاربر در هر استریم = user_stream_du
    <br>
    G. مدت زمان کل استریم = stream_du -->
    <br><hr>
</div>

<div dir="rtl">
خب حالا می تونیم امتیاز مورد نظر رو برای هر کاربر و کانال حساب کنیم. البته جاهایی که کسر 0/0 داشته باشیم مقدار این فرمول NaN میشه.<br>
    وقتی NaN میشه به این معنی هست که کاربر از قبل فعالیتی نداشته کلا و اولین باره که داره در اون کانال فعالیت میکنه. پس میتونیم اون سطر رو 0 در نظر بگیریم.
</div>

In [26]:
train = train.sort_values(by='start_time')

C_B = train['user_total_stream_per_channel']/train['user_total_stream']
C_B = C_B.fillna(0)

A_E = train['user_channel_du']/train['channel_total_stream_du_since_user_join']
A_E = A_E.fillna(0)

C_D = train['user_total_stream_per_channel']/train['channel_total_stream_since_user_join']
C_D = C_D.fillna(0)

train['channel_score'] = C_B + A_E + C_D

In [27]:
train['channel_score'].value_counts().sort_index()

0.000000    1363742
0.015010          1
0.015013          1
0.015678          1
0.018351          1
             ...   
2.976190          1
2.977778          1
2.981481          1
2.984848          1
3.000000       5198
Name: channel_score, Length: 875870, dtype: int64

In [28]:
train = train.sort_index()
train_copy = train.copy()
cols = ['user_stream_du', 'stream_du', 'stream_total_user','channel_total_stream_du_since_user_join',
        'user_channel_du', 'user_total_stream', 'user_total_stream_per_channel',
        'channel_total_stream', 'channel_total_stream_du','channel_total_stream_since_user_join',]
train.drop(cols, axis=1, inplace=True)

In [29]:
# حالا ما داریم با بازه های زمانی کار میکنیم، منطقا برای هر کاربر آخرین امتیازی که از کانال به دست میاد 
# بهتر داره آخرین وضعیت علاقه مندی کاربر به اون کانال رو مشخص میکنه
# پس یه کاری که میشه کرد اینکه بیایم برای هر کاربر از هر استریم، فقط آخرین استریم مربوط به اون کانال رو نگه داریم

train = train.sort_values(['user_id', 'start_time', 'streamer_name']).drop_duplicates(['user_id', 'streamer_name'], keep='last')
print(len(train['channel_score']))
train['channel_score'].value_counts().sort_index()

1363742


0.000000    877247
0.015678         1
0.018884         1
0.019541         1
0.019792         1
             ...  
2.967742         1
2.971429         1
2.973684         1
2.984848         1
3.000000       959
Name: channel_score, Length: 367532, dtype: int64

### Collaborative Filtering Using SVD

In [30]:
# حالا میتونیم بیایم یه ماتریس بسازیم که سطرهاش یوزرها هستن و ستون هاش اسم کانال ها
# اگه بین هر یوزر و کانال رابطه باشه و کاربر از قبل تو اون کانال فعال باشه امتیازی که برای اون کانال و برای اون کاربر محاسبه شده رو میذاریم
# هرچی امتیاز از 0 بزرگتر باشه به این معنیه که کاربر به کانال بیشتر علاقه منده
# صفر بودن امتیاز این رو نشون میده که کاربر فقط یک بار در استریم های اون کانال فعال بوده

print(len(train['user_id'].value_counts()))
print(len(train['streamer_name'].value_counts()))

99081
150867


In [31]:
# الان ماتریسی که باید ساخته بشه 99081 تا سطر داره و 150867 تا ستون! از طرفی ماتریس اسپارس هم هست. برای هندل کردن این بخش از 
# کتابخونه سای پای استفاده میکنیم، حالا چون اسم کانال ها رشته هست و ما برای ساختن ماتریس اندیس هایی که داریم عدد هستن
# میتونیم بیایم هر کانال رو مپ کنیم به یک عدد و برای این کار از لیبل اینکودر استفاده میشه کرد

In [32]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['streamer_name_label'] = le.fit_transform(train['streamer_name'])
train.head()

,user_id,stream_id,streamer_name,start_time,stop_time,channel_score,streamer_name_label
1,1,33846768288,alptv,166,169,0.000000,4953
5,1,33903958784,berkriptepe,734,737,0.000000,12902
16,1,34169481232,kendinemuzisyen,3600,3601,0.286017,66867
17,1,34185325968,unlostv,3739,3743,1.132898,137811
25,1,34236660832,elraenn,4314,4315,0.121575,36505


In [33]:
import scipy
from scipy.sparse import csr_matrix

user_item_matrix = csr_matrix((train['channel_score'], (train['user_id'], train['streamer_name_label'])))

print(user_item_matrix.shape)
print(len(user_item_matrix.data))

# results relalted to user with id 1
print('\nuser_id: 1')
print(user_item_matrix[1])

# results relalted to channel with id 1
print('\nchannel_id: 1')
print(user_item_matrix.T[1])

(100001, 150867)
1363742

user_id: 1
  (0, 4953)	0.0
  (0, 12902)	0.0
  (0, 36505)	0.12157481821660926
  (0, 37614)	0.0
  (0, 37985)	0.0
  (0, 49120)	1.0572100313479624
  (0, 50075)	0.0
  (0, 59879)	0.27403846153846156
  (0, 63507)	0.0749296408870877
  (0, 63611)	0.0
  (0, 66867)	0.2860169491525424
  (0, 85675)	0.4025607507960449
  (0, 95902)	0.4797659377887281
  (0, 106869)	0.19927536231884058
  (0, 134476)	0.7343358395989975
  (0, 137811)	1.132897603485839
  (0, 143842)	0.21090790023491077
  (0, 149396)	0.27017458028633223
  (0, 149653)	0.29627054361567634

channel_id: 1
  (0, 22714)	0.0


In [34]:
# این چیزی که اینجا توضیح میخوام بدم به روند حل سوال ربطی نداره، میتونید اسکیپ کنید 
# برای خودم سوال بود که چرا ابعاد ماتریس ساخته شده 100001 در 150867 هست
# ولی تعداد یوزرهامون در دیتاستی که داریم کمتر از این مقداره
# چیزی که فهمیدم این بود که چون آی دی یوزرها در دیتاست از 1 شروع میشه و یوزر با آی دی 100000 هم در دیتاست وجود داره
# در واقع اندیس هایی که برای یوزرها در نظر گرفته میشه از 0 هست تا 100000 پس تعدادش میشه 100001 تا

print(1 in train['user_id'])
print(100000 in train['user_id'])

True
True


In [35]:
# با توجه به ماتریسی که ساخته شده الان به ازای هر یوزر 150867 تا ستون که تعداد کانال هامون هست وجود خواهد داشت 
# میتونیم با روش اس وی دی این تعداد زیاد رو به کا بُعد کاهش بدیم

In [36]:
from scipy.sparse.linalg import svds

# The number of factors to factor the user-item matrix
k = 75

# Performs matrix factorization of the original user item matrix
U, S, VT = svds(user_item_matrix, k=k)

S = np.diag(S)

print(U.shape)
print(S.shape)
print(VT.shape)

(100001, 75)
(75, 75)
(75, 150867)


In [37]:
# از اینجا به بعد برای حل مسئله دو روش رو در نظر گرفتم.

# اولین راه که ساده ترین راه هست، اینکه اومدم ماتریس یوزر آیتم رو که بالاتر داشتم
# با استفاده از روش تجزیه اس وی دی به سه تا ماتریس شکستم و مجددا مقادیر امتیازها رو تخمین زدم.
# بعد برای هر یوزر کانال هایی که بیشترین امتیاز رو داشته باشن پینشهاد داده میشن.

# برای روش دوم اومدم از نزدیکی بین یوزرها استفاده کردم. یعنی یوزرهای مشابه بهم رو با استفاده از فاصله کسینوسی پیدا کردم 
# بعد اجتماعی از کانال هایی رو که یوزرهای مشابه بهشون علاقه مند بودن رو به افراد شبیه بهم پیشنهاد دادم

# در هر دو روش حواسم به این نکته بوده که کانالی که دارم به یوزرها پیشنهاد میدم جدید باشه

# یک روش سومی هم میتونه وجود داشته باشه، اونم اینکه کانال های مشابه بهم رو پیدا کنیم و اگه مثلا یوزری از کانال اول خوشش اومده باشه
# احتمالا از کانال مشابه به کانال اول هم خوشش خواهد اومد. این روش سوم رو اگر برسم پیاده سازی میکنم

### Solution 1

In [38]:
# الان ضرب پایین داره این رو مشخص میکنه که کاربر با آی دی 1 چقدر به هر کانال علاقه منده. 
# هر چقدر این عدد بزرگتر باشه، میزان علاقه مندی بیشتره
# در واقع داره همون ماتریس یوزر آیتم رو با تقریب برامون میسازه

np.dot(np.dot(U[1], S), VT)

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
        0.00000000e+00,  0.00000000e+00, -1.98840421e-05])

In [39]:
# تا اینجا میشه گفت که تقریبا مسئله حل شده
# فقط الان باید یه تابع بنویسیم که به کمکش آی دی یک یوزر رو بگیریم و تعداد مورد نظر کانالی رو که میخوایم (مثلا 10 تا) بهش پیشنهاد بدیم
# حواسمون هم هست که باید کانالی که پیشنهاد میدیم جدید باشه و کاربر از قبل ندیده باشه

In [40]:
def recommend_channel(user_id, n_channel=30, U=U, S=S, VT=VT, df=train, encoder=le):
    recommendations = []
    score = np.dot(np.dot(U[user_id], S), VT)
    best = np.argsort(-score)
    i = 0
    while len(recommendations) < n_channel:
        if not ((df['user_id'] == user_id) & (df['streamer_name_label'] == best[i])).any():
            recommendations.append(best[i])
        i+=1 
    return encoder.inverse_transform(recommendations)

### Test & Evaluation (Solution 1)

In [41]:
# شاید بهترین راه ارزیابی یک ریکامندر سیستم این باشه که ببینیم کاربر واقعی در طول زمان چطور با پیشنهاداتی که از طریق ما داده میشه رفتار میکنه
# پس یعنی میزان دقتی که اینجا به دست میاد و ارائه میشه خیلی درست نیست و صرفا یک راه ارزیابی سطحی و اولیه هست

<div dir="rtl">
برای ارزیابی نتایج به دست آمده، معیاری که در نظر گرفتم Hit Rate است که به شرح زیره:
    <br>
    + اول میام میبینم که کاربر مورد نظر در دیتای تست چند تا کانال جدید نسبت به دیتای آموزش در کل داشته
    <br>
    + بعد از بین کانال های جدیدی که در دیتای تست برای هر کاربر وجود داره، نگاه میکنم که آیا اون کانال رو به یوزر پیشنهاد دادم یا نه
    <br>
    + در نهایت نسبت تعداد یوزرهایی که بهشون حداقل یک پیشنهاد درست داده شده رو به تعداد کل یوزرها در نظر میگیرم
</div>
<br>
${HR = \frac{|U^L_{hit}|}{|U_{all}|}}$
<br><br>
${|U^L_{hit}|}$ is the number of users for which the correct answer is included in the top L recommendation list,
<br><br>
${|U_{all}|}$ is the total number of users in the test dataset.

In [42]:
# Count new channels for each user on test set
temp = test.sort_values(['user_id', 'start_time', 'streamer_name']).drop_duplicates(['user_id', 'streamer_name'], keep='last')
temp = temp[~temp.set_index(['user_id','streamer_name']).index.isin(train.set_index(['user_id','streamer_name']).index)]

In [192]:
# Call recommend_channel method for users who has at least one new channel
# users = temp.drop_duplicates(['user_id'], keep='last')
# users['recom'] = users['user_id'].apply(recommend_channel)
# usres.to_csv('users.csv')
# users.to_csv('30_ch_users.csv')

# از اونجایی که اجرای کد های بالا کمی زمانبر بود، پس از یک بار اجرا نتیجه به دست آمده رو
# در قالب یک فایل ذخیره کردم و برای اجراهای مجدد از فایل ذخیره شده استفاده میکنم

users = pd.read_csv('dataset/30_ch_users.csv', index_col=0)
print(users.shape)

(50108, 6)


In [193]:
def remove_new_line(x):
    x = x[1:-1]
    x = x.split(' ')
    x = [item.replace('\'', '') for item in x] 
    return [item.replace('\n', '') for item in x] 
    
users['recom'] = users['recom'].apply(remove_new_line)

In [194]:
# Merge temp and users df
merged_df = temp.set_index('user_id').merge(users.set_index('user_id')['recom'], left_index=True, right_index=True, how='left').set_index(temp.index)
merged_df.insert(0, 'user_id', temp['user_id'])

In [195]:
def count_predictions(x):
    return int(x[2] in x[5])
    
merged_df['prediction_recom'] = merged_df.apply(count_predictions, axis=1)

In [196]:
merged_df = merged_df.sort_values(['user_id', 'prediction_recom']).drop_duplicates(['user_id'], keep='last')

In [197]:
def hit_rate(df):
    hit_users = df[df['prediction_recom']==1].shape[0] # تعداد یوزرهایی که حداقل در یکی از کانال های پیشنهادی ما فعالیت کرده اند 
    all_users = df.shape[0] # تعداد کل یوزرها
    return hit_users / all_users

hit_rate(merged_df)

0.3222838668476092

### Solution 2

In [153]:
# خروجی تجزیه ماتریس به روش اس وی دی، سه ماتریس هست که اولین ماتریس در واقع داره ماتریس یوزرها رو نشون میده
# برای پیدا کردن یوزرهای مشابه از فاصله کسینوسی استفاده شده و 
# هر چقدر میزان شباهت به دست اومده برای دو کاربر به 1 نزدیک تر باشه، یعنی دو کاربر بیشتر بهم شبیه هستن

In [154]:
# با استفاده از تابع زیر میشه کا تا یوزر مشابه رو برای هر یوزر پیدا کرد. مقدار کا در اینجا 10 در نظر گرفته شده و
# حواسم بوده که خود کاربر رو در کاربرهای نزدیک نیارم

In [204]:
from sklearn.metrics.pairwise import cosine_similarity

def get_kn_users(user_id, k=10, U=U):
    similarity_score = cosine_similarity(U[user_id].reshape(1, -1), U)[0]
    kn_users = np.argsort(-similarity_score)[1: k+1]
    return kn_users

# kn_users = train.copy()
# kn_users = kn_users.drop_duplicates(['user_id'], keep='last')
# kn_users['kn_users'] = kn_users['user_id'].apply(get_kn_users)
# kn_users.to_csv('kn_users.csv')

# از اونجایی که اجرای کد های بالا کمی زمانبر بود، پس از یک بار اجرا نتیجه به دست آمده رو
# در قالب یک فایل ذخیره کردم و برای اجراهای مجدد از فایل ذخیره شده استفاده میکنم

kn_users = pd.read_csv('dataset/kn_users.csv', index_col=0)

In [205]:
def convert_to_int(x):
    x = x[1:-1]
    return np.fromstring(x, dtype=int, sep=' ')

kn_users['kn_users'] = kn_users['kn_users'].apply(convert_to_int)
kn_users.head()

,user_id,stream_id,streamer_name,start_time,stop_time,channel_score,streamer_name_label,kn_users
44,1,34347669376,grimnax,5415,5419,1.057210,49120,"[24561, 89853, 20497, 56288, 57420, 80246, 303..."
81,2,34294691376,sal_gu,4862,4871,0.753011,112765,"[87921, 66966, 33177, 34340, 22104, 46723, 932..."
101,3,34320336432,dakillzor,5181,5182,0.000000,26827,"[64577, 56112, 36299, 63580, 76238, 69295, 423..."
161,4,34247687792,script,4425,4429,0.509518,114530,"[21052, 95830, 56288, 718, 30315, 57420, 89867..."
165,5,34165772528,nick28t,3595,3596,1.154241,92366,"[58912, 55308, 53968, 67876, 97317, 63914, 798..."


In [206]:
# طبق نتایج به دست امده، کاربر با آی دی 1 بیشترین شباهت را به کاربر با آی دی 24561 دارد. 
# برای بررسی جزئی تر، مورد زیر بعنوان نمونه ارائه شده است

user1 = train[(train['user_id']==1)]
user24561 = train[(train['user_id']==24561)]
print(user1.shape[0])
print(user24561.shape[0])
print(pd.merge(user1, user24561, how='inner', on=['streamer_name']).sort_values('streamer_name').shape[0])

19
15
8


In [207]:
# شباهت دو کاربر بهم دیگه به این معنیه که تعداد کانال های مشترکشون بیشتر بوده
# و اگه یکی از دو کاربر از کانال های دیگه خوشش بیاد احتمالا کاربر اول هم از اون کانال ها خوشش خواهد اومد
# می تونیم از بین 10 تا کاربر نزدیک برای هر یوزر 10 تا از کانال هایی رو که در اون فعالیت داشتن رو خروجی بدیم 

In [208]:
train = train.sort_values(['user_id', 'channel_score'], ascending=[1, 0])

In [209]:
def recommend_channels(user_id, k=30, df1=kn_users, df2=train):
    result = list()
    similar_users = df1[df1['user_id']==user_id]['kn_users'].values
    if len(similar_users) > 0:
        similar_users = similar_users[0]
        user_channel = df2[df2['user_id']==user_id]['streamer_name'].values
        for user in similar_users:
            similar_channel = df2[df2['user_id']==user]['streamer_name'].values
            for ch in similar_channel:
                if ch not in user_channel and ch not in result:
                    result.append(ch)
                if len(result) == k:
                    return result
    return result

In [210]:
# برای نمونه کانال های پیشنهادی به کاربر با آی دی 3 به صورت زیر است

recommend_channels(3)

['dre3dd',
 'paplovag',
 'cucu0015',
 'fyrexxx',
 'yeahunter',
 'blyyyplays',
 'pingvin_harcos',
 'kd_ddave',
 'zsozeatyaofficial',
 'kriszhadvice',
 'ninja',
 'soaryn',
 'bturbo',
 'tsm_viss',
 'amouranth']

In [226]:
# برای همه کاربران کانال های پیشنهادی بررسی می شود

# kn_users['recom']= kn_users['user_id'].apply(recommend_channels)
# kn_users.to_csv('kn_users_recom.csv')

kn_users = pd.read_csv('dataset/kn_users_recom.csv', index_col=0)

In [227]:
def to_str_array(x):
    x = x[1:-1]
    x = x.split(' ')
    x = [item.replace('\'', '') for item in x] 
    return [item.replace(',', '') for item in x] 
    
kn_users['recom'] = kn_users['recom'].apply(to_str_array)


### Test & Evaluation (Solution 2)

In [228]:
# معیار ارزیابی این روش نیز مشابه روش قبلی است

In [229]:
merged_df = temp.set_index('user_id').merge(kn_users.set_index('user_id')['recom'], left_index=True, right_index=True, how='left').set_index(temp.index)
merged_df.insert(0, 'user_id', temp['user_id'])

In [230]:
def count_predictions(x):
    if x[5] is np.nan:
        return 0
    return int(x[2] in x[5])
    
merged_df['prediction_recom'] = merged_df.apply(count_predictions, axis=1)

In [231]:
merged_df = merged_df.sort_values(['user_id', 'prediction_recom']).drop_duplicates(['user_id'], keep='last')

In [232]:
def hit_rate(df):
    hit_users = df[df['prediction_recom']==1].shape[0] # تعداد یوزرهایی که حداقل در یکی از کانال های پیشنهادی ما فعالیت کرده اند 
    all_users = df.shape[0] # تعداد کل یوزرها
    return hit_users / all_users

hit_rate(merged_df)

0.288576674383332

### Conclusion

| Solution                                      | Recommended Channels   | Hit Rate   |
|:----------------------------------------------|:----------------------:|:----------:|
| Reconstruction of User/Item Matrix using SVD  | 30                     | 32%        |
| Finding Silimar Users using Cosine Similarity | 30                     | 29%        |
| Reconstruction of User/Item Matrix using SVD  | 10                     | 21%        |
| Finding Silimar Users using Cosine Similarity | 10                     | 15%        |

In [233]:
# Thank you :)